In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# FUNÇÕES:
def filtro_media_nao_causal(signal, janela = 128):
    
    M = int(janela/2)    
    modified_signal = np.hstack(( np.zeros(M, dtype=float), signal, np.zeros(M, dtype=float) ))    
    mean_signal = np.zeros(len(signal), dtype=float) # Inicialização do array
    
    for n in range(M, len(signal)+M):        
        mean_signal[n-M] = modified_signal[(n-M):(n+M)].mean()
        
    filtered_signal = signal - mean_signal 
    return filtered_signal, mean_signal   


In [4]:
os.chdir('root')
file_names = os.listdir('root/Raw_Data/')

for file_name in file_names:
    df = pd.read_excel('Raw_Data/' + file_name, names = ['bitF', 'bitT', 'sup', 'supr', 'emp1', 'emp2', 'emp3', 'raio', 'nle', 'nld', 'ale', 'ald', 'tipo', 'kmi', 'kmf', 'linha', 'eh', 'superv', 'inspec', 'ano'])
    
    # FILTRO DOS DADOS:
    filter_data = df.drop(['bitT', 'sup', 'emp1', 'emp2', 'raio'], axis = 1) # Eliminando informações irrelevantes
    signals_used = filter_data.drop(['tipo', 'kmi', 'kmf', 'linha', 'eh', 'superv', 'ano'], axis = 1) # Eliminando informações descritivas
    signals_used.columns = ['1', '2', '3', '4', '5', '6', '7', '8'] # Mantendo apenas os 7 sinais + número da inspeção
        
    # PROCESSAMENTOS DOS DADOS:
    for n_inspec in range(3,7): # Inspeções 3, 4, 5 e 6
        
        for sinal in list(map(str, list(range(1,8)))):
            
            list_inspec = (signals_used['8'] == int(n_inspec)).to_numpy().nonzero()[0] # Criando índices de uma inspeção específica
            signal_used = signals_used[sinal].to_numpy()
            signal_used = signal_used[list_inspec] # Selecionando os dados referente à uma única inspeção
            
            # Inicialização de parâmetros ---------------------
            energy = np.empty(shape=len(signal_used))
            variance = np.empty(shape=len(signal_used))
            amplitude_maxmin = np.empty(shape=len(signal_used))
            zero_cross_rating = np.empty(shape=len(signal_used))            
            if sinal == '1':
                spatial_parameters = np.empty(shape=len(signal_used))
            # --------------------------------------------------
            
            [filtered_signal, mean_signal] = filtro_media_nao_causal(signal_used, 128) 
            extended_signal = np.hstack(( np.zeros(128, dtype=float), signal_used, np.zeros(128, dtype=float) ))
            extended_signal_no_media = np.hstack(( np.zeros(128, dtype=float), filtered_signal, np.zeros(128, dtype=float) ))

            for n_desloc in range(0,len(signal_used)):
                 
                windown_selec0 = extended_signal[n_desloc:n_desloc+128];
                windown_selec1 = extended_signal_no_media[n_desloc:n_desloc+128];

                if sinal == '2': # Sinal de superelevação tem a média subtraída.
                    windown_selec = windown_selec1;
                else:
                    windown_selec = windown_selec0;

                # ENCONTRANDO OS PARÂMETROS:                    
                # Energia do sinal dentro da janela selecionada
                energy[n_desloc] = sum(windown_selec**2)
                # Variância do sinal dentro da janela selecionada
                variance[n_desloc] = np.var(windown_selec)
                # Amplitude máxima do sinal dentro da janela
                amplitude_maxmin[n_desloc] = max(windown_selec) - min(windown_selec)
                # Taxa de Cruzamento por zeros dentro da janela selecionada

            
            # CONCATENAÇÃO DOS PARÂMETROS
            spatial_parameters = np.vstack((spatial_parameters, energy, variance, amplitude_maxmin))
          

# Descomentar última linha para gerar as saídas transformadas #        
        base_name = ['Base_E_' + file_name[4:10] + '_' + str(n_inspec) + '.npy']
#         np.save(base_name, spatial_parameters)